In [1]:
import numpy as np

In [79]:
network = 'data/I210'

In [80]:
graph = np.loadtxt(network + '_net.csv', delimiter=',', skiprows=1)
demand = np.loadtxt(network + '_od.csv', delimiter=',', skiprows=1)

In [81]:
# in the case where there is only one o-d, then demand is interpret as a single row and not as a matrix
try:
    demand.shape[1]
except:
    demand = np.array([demand])
nb_ods = int(demand.shape[0])

In [82]:
# graph_dict gives the line of the graph matrix corresponding to the destination d and the origin o
graph_dict = {}
for i in range(graph.shape[0]):
    try: 
        graph_dict[int(graph[i][1])]
    except:
        graph_dict[int(graph[i][1])] = {}
    graph_dict[int(graph[i][1])][int(graph[i][2])] = int(graph[i][0])

In [83]:
print(demand.shape)

(1, 3)


In [84]:
def travel_time(f):
    return graph[:,3] + graph[:,4]*f + graph[:,5]*(f**2) + graph[:,6]*(f**3) + graph[:,7]*(f**4)

In [85]:
def hessian(f):
    return graph[:,4] + 2*graph[:,5]*(f) + 3*graph[:,6]*(f**2) + 4*graph[:,7]*(f**3)

In [86]:
nb_links = int(np.max(graph[:,0])+1)
nb_nodes = int(max(np.max(graph[:,1]), np.max(graph[:,2]))+1)
print(nb_nodes)

21


In [87]:
print(demand)

[[1.9e+01 2.0e+01 2.5e+04]]


In [88]:
# graph_dict[3] = {0: 5}
print(graph_dict)

{1: {2: 0, 7: 1}, 2: {3: 2, 8: 3}, 3: {4: 4, 9: 5}, 4: {5: 6, 10: 7}, 5: {6: 8, 11: 9}, 6: {12: 10}, 7: {1: 11, 8: 12, 13: 13}, 8: {2: 14, 9: 15, 14: 16}, 9: {3: 17, 10: 18, 15: 19}, 10: {4: 20, 11: 21, 16: 22}, 11: {5: 23, 12: 24, 17: 25}, 12: {6: 26, 18: 27, 20: 39}, 13: {7: 28, 14: 29}, 14: {8: 30, 15: 31}, 15: {9: 32, 16: 33}, 16: {10: 34, 17: 35}, 17: {11: 36, 18: 37}, 18: {12: 38}, 19: {7: 40}}


In [89]:
## DO THE CASE WHERE THERE IS A LOOP IN THE NETWORK
## USE A CLASS, YOU CANNOT USE LOCAL SET IN PYTHON

def add_node(o, d, test, o_tmp, tab):
    # print("If I want to access o=" + str(o_tmp) + " from d=" + str(d) + ", i can go through n="+ str(o))
    if test[o][d]==-1:
        test[o][d] = set()
    test[o][d].add(o_tmp)
    # print("I've just put test[" + str(o_tmp) + "][" + str(d) + "]=" + str(o))
    print(tab + "test[" + str(o) + "][" + str(d) + "]=" + str(test[o][d]))
    # print()

def update(o, d, test, pred, visited, tab):
    # print("I am in the update corresponding to node " + str(d))
    # print("First, i need to put the predessor of d:" + str(predecessor))
    # add_node(o, d, test, o)
    for o_pred in pred:
        # print("Let do it for o:" + str(o_tmp))
        add_node(o_pred, d, test, o, tab)
    
    predecessor_tmp = pred.copy() # HERE THE COPY IS NOT A REAL COPY! STILL HAVE THE SAME MEMORY
    predecessor_tmp.add(d)
    print(tab + str(d) + "; pred: " + str(pred))
    try:
        # print("Now, let do it again for the node after d=" + str(d))
        # print("The node which follow d are:")
        for d_tmp in graph_dict[d].keys():
            print(tab + str(d) + "->" + str(d_tmp))
            if d_tmp not in visited:
                visited.add(d_tmp)
                update(d, d_tmp, test, predecessor_tmp, visited.copy(), tab + "   " + str(d) + ":   ")
    except:
        return

In [90]:
test = np.array(np.zeros(shape=(nb_nodes, nb_nodes))-1,dtype=object)
for o in demand[:,0]:
    o = int(o)
# for o in graph_dict.keys():
    # print()
    # print("************************************")
    # print("I consider " + str(o))
    predecessor = set()
    predecessor.add(o)
    for d in graph_dict[o].keys():
        print("I will update " + str(d))
        # print("I need to put test[" + str(o) + "][" + str(d) + "]=" + str(o))
        # print("test[" + str(o) + "][" + str(d) + "]=" + str(test[o][d]))
        update(o, d, test, predecessor, predecessor.copy(), "")

I will update 7
test[19][7]={19}
7; pred: {19}
7->1
   7:   test[19][1]={7}
   7:   test[7][1]={7}
   7:   1; pred: {19, 7}
   7:   1->2
   7:      1:   test[1][2]={1}
   7:      1:   test[19][2]={1}
   7:      1:   test[7][2]={1}
   7:      1:   2; pred: {1, 19, 7}
   7:      1:   2->3
   7:      1:      2:   test[1][3]={2}
   7:      1:      2:   test[2][3]={2}
   7:      1:      2:   test[19][3]={2}
   7:      1:      2:   test[7][3]={2}
   7:      1:      2:   3; pred: {1, 2, 19, 7}
   7:      1:      2:   3->4
   7:      1:      2:      3:   test[1][4]={3}
   7:      1:      2:      3:   test[2][4]={3}
   7:      1:      2:      3:   test[3][4]={3}
   7:      1:      2:      3:   test[7][4]={3}
   7:      1:      2:      3:   test[19][4]={3}
   7:      1:      2:      3:   4; pred: {1, 2, 3, 7, 19}
   7:      1:      2:      3:   4->5
   7:      1:      2:      3:      4:   test[1][5]={4}
   7:      1:      2:      3:      4:   test[2][5]={4}
   7:      1:      2:      3:      4: 

   7:      1:      7:      8:      14:   test[14][15]={14}
   7:      1:      7:      8:      14:   test[19][15]={9, 14}
   7:      1:      7:      8:      14:   15; pred: {1, 7, 8, 14, 19}
   7:      1:      7:      8:      14:   15->9
   7:      1:      7:      8:      14:   15->16
   7:      1:      7:      8:      14:      15:   test[1][16]={10, 15}
   7:      1:      7:      8:      14:      15:   test[19][16]={10, 15}
   7:      1:      7:      8:      14:      15:   test[7][16]={10, 15}
   7:      1:      7:      8:      14:      15:   test[8][16]={10, 15}
   7:      1:      7:      8:      14:      15:   test[14][16]={15}
   7:      1:      7:      8:      14:      15:   test[15][16]={15}
   7:      1:      7:      8:      14:      15:   16; pred: {1, 19, 7, 8, 14, 15}
   7:      1:      7:      8:      14:      15:   16->10
   7:      1:      7:      8:      14:      15:      16:   test[16][10]={16}
   7:      1:      7:      8:      14:      15:      16:   test[1][10]={16, 9,

   7:      8:   9; pred: {8, 19, 7}
   7:      8:   9->3
   7:      8:      9:   test[8][3]={9, 2}
   7:      8:      9:   test[9][3]={9}
   7:      8:      9:   test[19][3]={9, 2}
   7:      8:      9:   test[7][3]={9, 2}
   7:      8:      9:   3; pred: {8, 9, 19, 7}
   7:      8:      9:   3->4
   7:      8:      9:      3:   test[3][4]={3}
   7:      8:      9:      3:   test[7][4]={10, 3}
   7:      8:      9:      3:   test[8][4]={10, 3}
   7:      8:      9:      3:   test[9][4]={10, 3}
   7:      8:      9:      3:   test[19][4]={10, 3}
   7:      8:      9:      3:   4; pred: {3, 7, 8, 9, 19}
   7:      8:      9:      3:   4->5
   7:      8:      9:      3:      4:   test[3][5]={11, 4}
   7:      8:      9:      3:      4:   test[19][5]={11, 4}
   7:      8:      9:      3:      4:   test[4][5]={4}
   7:      8:      9:      3:      4:   test[7][5]={11, 4}
   7:      8:      9:      3:      4:   test[8][5]={11, 4}
   7:      8:      9:      3:      4:   test[9][5]={11, 4}
   

In [91]:
print(test)

[[-1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0
  -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0]
 [-1.0 -1.0 {1} {9, 2} {10, 3} {11, 4} {12, 5} {1} {2, 7} {8, 3, 15}
  {16, 9, 4} {17, 10, 5} {18, 11, 6} {7} {8, 13} {9, 14} {10, 15}
  {16, 11} {17, 12} -1.0 {12}]
 [-1.0 -1.0 -1.0 {2} {10, 3} {11, 4} {12, 5} -1.0 {2} {8, 3} {16, 9, 4}
  {17, 10, 5} {18, 11, 6} -1.0 {8} {9, 14} {10, 15} {16, 11} {17, 12}
  -1.0 {12}]
 [-1.0 -1.0 -1.0 -1.0 {3} {11, 4} {12, 5} -1.0 -1.0 {3} {9, 4}
  {17, 10, 5} {18, 11, 6} -1.0 -1.0 {9} {10, 15} {16, 11} {17, 12} -1.0
  {12}]
 [-1.0 -1.0 -1.0 -1.0 -1.0 {4} {12, 5} -1.0 -1.0 -1.0 {4} {10, 5}
  {18, 11, 6} -1.0 -1.0 -1.0 {10} {16, 11} {17, 12} -1.0 {12}]
 [-1.0 -1.0 -1.0 -1.0 -1.0 -1.0 {5} -1.0 -1.0 -1.0 -1.0 {5} {11, 6} -1.0
  -1.0 -1.0 -1.0 {11} {17, 12} -1.0 {12}]
 [-1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 {6} -1.0
  -1.0 -1.0 -1.0 -1.0 {12} -1.0 {12}]
 [-1.0 {7} {8, 1} {9, 2} {10, 3} {11, 4} {12, 5} {1, 13} {2, 7}
  {8, 3, 

In [15]:
test = np.array(np.zeros(shape=(nb_nodes, nb_nodes))-1,dtype=object)
for o in graph_dict.keys():
    for d in graph_dict[o].keys():
        if test[o-1][d-1]==-1:
            test[o-1][d-1] = set()
        test[o-1][d-1].add(o-1)# += {o-1}
    

In [16]:
print(graph_dict[19])

KeyError: 19

In [140]:
print(test[18][6])

{18}
